# Make a bunch of logos

Try a couple of sizes
- 14 by 14 
- 30 by 30

In [401]:
import numpy as np
import pandas as pd
import requests as re
from PIL import Image
from scipy.ndimage import convolve
from io import BytesIO

import os
import time
from RGBMatrixEmulator import RGBMatrix, RGBMatrixOptions

In [374]:
mlb_logos = pd.read_csv("MLB_Colors_Logos.csv")

In [375]:
mlb_logos

,team_name,team_location,team_mascot,team_abbr,team_id_num,team_logo_espn,team_scoreboard_logo_espn,team_cap_logo_on_light,team_cap_logo_on_dark,team_color,team_color2,team_color3,team_color4,team_color5,team_league,team_division,team_division_location,team_league_logo,mlb_logo
0,Arizona Diamondbacks,Arizona,Diamondbacks,AZ,109.0,https://a.espncdn.com/i/teamlogos/mlb/500/ari.png,https://a.espncdn.com/i/teamlogos/mlb/500/scor...,https://www.mlbstatic.com/team-logos/team-cap-...,https://www.mlbstatic.com/team-logos/team-cap-...,#A71930,#E3D4AD,#000000,#30CED8,#FFFFFF,NL,NL West,West,https://a.espncdn.com/combiner/i?img=/i/teamlo...,https://a.espncdn.com/combiner/i?img=/i/teamlo...
1,Atlanta Braves,Atlanta,Braves,ATL,144.0,https://a.espncdn.com/i/teamlogos/mlb/500/atl.png,https://a.espncdn.com/i/teamlogos/mlb/500/scor...,https://www.mlbstatic.com/team-logos/team-cap-...,https://www.mlbstatic.com/team-logos/team-cap-...,#CE1141,#13274F,#EAAA00,#FFFFFF,NaN,NL,NL East,East,https://a.espncdn.com/combiner/i?img=/i/teamlo...,https://a.espncdn.com/combiner/i?img=/i/teamlo...
2,Baltimore Orioles,Baltimore,Orioles,BAL,110.0,https://a.espncdn.com/i/teamlogos/mlb/500/bal.png,https://a.espncdn.com/i/teamlogos/mlb/500/scor...,https://www.mlbstatic.com/team-logos/team-cap-...,https://www.mlbstatic.com/team-logos/team-cap-...,#DF4601,#000000,NaN,NaN,NaN,AL,AL East,East,https://a.espncdn.com/combiner/i?img=/i/teamlo...,https://a.espncdn.com/combiner/i?img=/i/teamlo...
3,Boston Red Sox,Boston,Red Sox,BOS,111.0,https://a.espncdn.com/i/teamlogos/mlb/500/bos.png,https://a.espncdn.com/i/teamlogos/mlb/500/scor...,https://www.mlbstatic.com/team-logos/team-cap-...,https://www.mlbstatic.com/team-logos/team-cap-...,#BD3039,#0C2340,#FFFFFF,NaN,NaN,AL,AL East,East,https://a.espncdn.com/combiner/i?img=/i/teamlo...,https://a.espncdn.com/combiner/i?img=/i/teamlo...
4,Chicago Cubs,Chicago,Cubs,CHC,112.0,https://a.espncdn.com/i/teamlogos/mlb/500/chc.png,https://a.espncdn.com/i/teamlogos/mlb/500/scor...,https://www.mlbstatic.com/team-logos/team-cap-...,https://www.mlbstatic.com/team-logos/team-cap-...,#0E3386,#CC3433,NaN,NaN,NaN,NL,NL Central,Central,https://a.espncdn.com/combiner/i?img=/i/teamlo...,https://a.espncdn.com/combiner/i?img=/i/teamlo...
5,Chicago White Sox,Chicago,White Sox,CWS,145.0,https://a.espncdn.com/i/teamlogos/mlb/500/cws.png,https://a.espncdn.com/i/teamlogos/mlb/500/scor...,https://www.mlbstatic.com/team-logos/team-cap-...,https://www.mlbstatic.com/team-logos/team-cap-...,#27251F,#C4CED4,NaN,NaN,NaN,AL,AL Central,Central,https://a.espncdn.com/combiner/i?img=/i/teamlo...,https://a.espncdn.com/combiner/i?img=/i/teamlo...
6,Cincinnati Reds,Cincinnati,Reds,CIN,113.0,https://a.espncdn.com/i/teamlogos/mlb/500/cin.png,https://a.espncdn.com/i/teamlogos/mlb/500/scor...,https://www.mlbstatic.com/team-logos/team-cap-...,https://www.mlbstatic.com/team-logos/team-cap-...,#C6011F,#000000,NaN,NaN,NaN,NL,NL Central,Central,https://a.espncdn.com/combiner/i?img=/i/teamlo...,https://a.espncdn.com/combiner/i?img=/i/teamlo...
7,Cleveland Guardians,Cleveland,Guardians,CLE,114.0,https://a.espncdn.com/i/teamlogos/mlb/500/cle.png,https://a.espncdn.com/i/teamlogos/mlb/500/scor...,https://www.mlbstatic.com/team-logos/team-cap-...,https://www.mlbstatic.com/team-logos/team-cap-...,#0C2340,#E31937,NaN,NaN,NaN,AL,AL Central,Central,https://a.espncdn.com/combiner/i?img=/i/teamlo...,https://a.espncdn.com/combiner/i?img=/i/teamlo...
8,Colorado Rockies,Colorado,Rockies,COL,115.0,https://a.espncdn.com/i/teamlogos/mlb/500/col.png,https://a.espncdn.com/i/teamlogos/mlb/500/scor...,https://www.mlbstatic.com/team-logos/team-cap-...,https://www.mlbstatic.com/team-logos/team-cap-...,#33006F,#C4CED4,#000000,NaN,NaN,NL,NL West,West,https://a.espncdn.com/combiner/i?img=/i/teamlo...,https://a.espncdn.com/combiner/i?img=/i/teamlo...
9,Detroit Tigers,Detroit,Tigers,DET,116.0,https://a.espncdn.com/i/teamlogos/mlb/500/det.png,https://a.espncdn.com/i/teamlogos/mlb/500/scor...,https://www.mlbsta

In [376]:
mlb_logos.team_abbr.values

array(['AZ', 'ATL', 'BAL', 'BOS', 'CHC', 'CWS', 'CIN', 'CLE', 'COL',
       'DET', 'HOU', 'KC', 'LAA', 'LAD', 'MIA', 'MIL', 'MIN', 'NYM',
       'NYY', 'OAK', 'PHI', 'PIT', 'SD', 'SF', 'SEA', 'STL', 'TB', 'TEX',
       'TOR', 'WSH', 'NL', 'AL', 'MLB'], dtype=object)

In [412]:
base_img_url = mlb_logos.loc[mlb_logos["team_abbr"] == "MLB", "team_scoreboard_logo_espn"].values[0]


In [413]:
base_img_url

'https://a.espncdn.com/combiner/i?img=/i/teamlogos/leagues/500/mlb.png'

In [429]:
# IDK which size over here!
base_width= 14
img_resp = re.get(base_img_url)

img = Image.open(BytesIO(img_resp.content))

In [430]:
img.show()

In [386]:
wpercent = (base_width / float(img.size[0]))
hsize = int((float(img.size[1]) * float(wpercent)))
small_img = img.resize((base_width, hsize), Image.Resampling.LANCZOS)
#img.save('somepic.jpg')

In [434]:
small_img

In [435]:
img_as_array = np.array(small_img.convert('RGBA'))

print(img_as_array.shape)

(30, 30, 4)


In [436]:
Image.fromarray(img_as_array, 'RGBA')

## Hmm I think I need to collapse the three channels

And I also need to do the max instead of a convolution?

In [437]:
def drop_pixels(image, num_neighbors):
    image_copy = np.copy(image)
    print(np.sum(image_copy), image_copy.shape)
    
    # only do this for the RGB, not the A
    non_zero_pixels = np.where(np.sum(image_copy[:, :, :3], axis=2) > 0, 1, 0)
    
    # count the number of adjacent pixels
    neighborhood_filter = np.array([[0, 1, 0],
                                    [1, 0, 1],
                                    [0, 1, 0]])

    neighbors_count = convolve(non_zero_pixels, neighborhood_filter, mode='constant', cval=0.0)
    
    mask = neighbors_count > num_neighbors

    print("how many pixel with more than {} neighbors".format(num_neighbors), np.sum(mask), mask.shape)
    
    image_copy[~mask, :3] = 0

    print(np.sum(image_copy), image_copy.shape)
    
    return image_copy


In [438]:
img_as_array

array([[[0, 0, 0, 0],
        [0, 0, 0, 0],
        [0, 0, 0, 0],
        ...,
        [0, 0, 0, 0],
        [0, 0, 0, 0],
        [0, 0, 0, 0]],

       [[0, 0, 0, 0],
        [0, 0, 0, 0],
        [0, 0, 0, 0],
        ...,
        [0, 0, 0, 0],
        [0, 0, 0, 0],
        [0, 0, 0, 0]],

       [[0, 0, 0, 0],
        [0, 0, 0, 0],
        [0, 0, 0, 0],
        ...,
        [0, 0, 0, 0],
        [0, 0, 0, 0],
        [0, 0, 0, 0]],

       ...,

       [[0, 0, 0, 0],
        [0, 0, 0, 0],
        [0, 0, 0, 0],
        ...,
        [0, 0, 0, 0],
        [0, 0, 0, 0],
        [0, 0, 0, 0]],

       [[0, 0, 0, 0],
        [0, 0, 0, 0],
        [0, 0, 0, 0],
        ...,
        [0, 0, 0, 0],
        [0, 0, 0, 0],
        [0, 0, 0, 0]],

       [[0, 0, 0, 0],
        [0, 0, 0, 0],
        [0, 0, 0, 0],
        ...,
        [0, 0, 0, 0],
        [0, 0, 0, 0],
        [0, 0, 0, 0]]], dtype=uint8)

In [439]:
Image.fromarray(drop_pixels(img_as_array, 2))

356805 (30, 30, 4)
how many pixel with more than 2 neighbors 446 (30, 30)
305068 (30, 30, 4)


In [440]:
Image.fromarray(drop_pixels(img_as_array, 2))

356805 (30, 30, 4)
how many pixel with more than 2 neighbors 446 (30, 30)
305068 (30, 30, 4)


In [441]:
file_path = "./assets/team_logos/"

for team_abbr in mlb_logos.team_abbr.values:
    print(team_abbr)
    
    os.makedirs(file_path + team_abbr +"/", exist_ok=True)
    
    base_img_url = mlb_logos.loc[mlb_logos["team_abbr"] == team_abbr, "team_scoreboard_logo_espn"].values[0]

    print(base_img_url)
    
    img_resp = re.get(base_img_url)
    img = Image.open(BytesIO(img_resp.content))
    
    # IDK which size over here!
    for base_width in [14, 30]:
        
        wpercent = (base_width / float(img.size[0]))
        hsize = int((float(img.size[1]) * float(wpercent)))
        small_img = img.resize((base_width, hsize), Image.Resampling.LANCZOS)

        img_as_array = np.array(small_img.convert('RGBA'))

        for neighbors in [1, 2, 3]:
            cleaned_img = Image.fromarray(drop_pixels(img_as_array, neighbors))

            file_name = "{}-w{}n{}.png".format(team_abbr, base_width, neighbors)
            
            cleaned_img.save(file_path + team_abbr + "/" + file_name, "PNG")
    

AZ
https://a.espncdn.com/i/teamlogos/mlb/500/scoreboard/ari.png
55447 (14, 14, 4)
how many pixel with more than 1 neighbors 134 (14, 14)
52586 (14, 14, 4)
55447 (14, 14, 4)
how many pixel with more than 2 neighbors 108 (14, 14)
45584 (14, 14, 4)
55447 (14, 14, 4)
how many pixel with more than 3 neighbors 71 (14, 14)
33865 (14, 14, 4)
248309 (30, 30, 4)
how many pixel with more than 1 neighbors 533 (30, 30)
230670 (30, 30, 4)
248309 (30, 30, 4)
how many pixel with more than 2 neighbors 406 (30, 30)
199620 (30, 30, 4)
248309 (30, 30, 4)
how many pixel with more than 3 neighbors 308 (30, 30)
157193 (30, 30, 4)
ATL
https://a.espncdn.com/i/teamlogos/mlb/500/scoreboard/atl.png
40399 (14, 14, 4)
how many pixel with more than 1 neighbors 149 (14, 14)
39509 (14, 14, 4)
40399 (14, 14, 4)
how many pixel with more than 2 neighbors 117 (14, 14)
36258 (14, 14, 4)
40399 (14, 14, 4)
how many pixel with more than 3 neighbors 82 (14, 14)
31072 (14, 14, 4)
155124 (30, 30, 4)
how many pixel with more than

## Do some overriding certain methods so that if the alpha level is 0, it just skips writing that one

Relating to issue #3 

In [446]:
# read in a sample file
file_path = "./assets/team_logos/"
team_abbr = "CWS"


file_name = "{}-w{}n{}.png".format(team_abbr, 30, 2)

img = Image.open(file_path + team_abbr + "/" + file_name)
img

In [447]:

np.array(img.convert('RGBA'))

array([[[0, 0, 0, 0],
        [0, 0, 0, 0],
        [0, 0, 0, 0],
        ...,
        [0, 0, 0, 0],
        [0, 0, 0, 0],
        [0, 0, 0, 0]],

       [[0, 0, 0, 0],
        [0, 0, 0, 0],
        [0, 0, 0, 0],
        ...,
        [0, 0, 0, 0],
        [0, 0, 0, 0],
        [0, 0, 0, 0]],

       [[0, 0, 0, 0],
        [0, 0, 0, 0],
        [0, 0, 0, 0],
        ...,
        [0, 0, 0, 0],
        [0, 0, 0, 0],
        [0, 0, 0, 0]],

       ...,

       [[0, 0, 0, 0],
        [0, 0, 0, 0],
        [0, 0, 0, 0],
        ...,
        [0, 0, 0, 0],
        [0, 0, 0, 0],
        [0, 0, 0, 0]],

       [[0, 0, 0, 0],
        [0, 0, 0, 0],
        [0, 0, 0, 0],
        ...,
        [0, 0, 0, 0],
        [0, 0, 0, 0],
        [0, 0, 0, 0]],

       [[0, 0, 0, 0],
        [0, 0, 0, 0],
        [0, 0, 0, 0],
        ...,
        [0, 0, 0, 0],
        [0, 0, 0, 0],
        [0, 0, 0, 0]]], dtype=uint8)

In [448]:
img_as_array = np.array(img.convert('RGBA'))

In [450]:
img_as_array[10:20, 10:20, :]

array([[[  8,   3,   4, 253],
        [ 42,  39,  40, 255],
        [144, 142, 143, 255],
        [209, 208, 208, 255],
        [127, 125, 126, 255],
        [ 36,  32,  33, 255],
        [ 23,  19,  20, 255],
        [ 24,  20,  21, 251],
        [ 56,  53,  54, 254],
        [187, 186, 186, 255]],

       [[ 92,  89,  90, 255],
        [ 32,  28,  29, 253],
        [  4,   0,   1, 252],
        [ 54,  51,  52, 255],
        [219, 218, 218, 255],
        [216, 215, 215, 255],
        [ 91,  88,  89, 255],
        [ 27,  23,  24, 255],
        [ 22,  18,  19, 250],
        [113, 110, 110, 254]],

       [[212, 212, 212, 203],
        [171, 169, 169, 255],
        [113, 110, 111, 255],
        [ 19,  14,  15, 255],
        [ 99,  97,  97, 255],
        [255, 255, 255, 255],
        [255, 255, 255, 255],
        [101,  99,  99, 255],
        [ 15,  11,  12, 252],
        [ 83,  80,  81, 255]],

       [[  0,   0,   0,   0],
        [255, 255, 255,  54],
        [232, 232, 232, 173],
    

Ok so I think this is good maybe?

## Override the set image and set pixel def! 

In [451]:
img

In [453]:
pixels = [pixel for pixel in img.getdata()]

In [456]:
(*pixels[0])

SyntaxError: cannot use starred expression here (4135990850.py, line 1)

In [ ]:
def SetImage(self, image, offset_x=0, offset_y=0, *other):
    self.__sync_canvas()
    self.canvas.SetImage(image, offset_x, offset_y, *other)
    self.SwapOnVSync(self.canvas)

In [460]:
def SetPixelAlpha(x, y, r, g, b, a):
    print(a)

In [461]:
SetPixel(0, 0, *pixels[0])

0


In [ ]:
def _NewSetPixel(self, x, y, r, g, b, a, threshold=10):
    print("new set pixel")
    if self.display_adapter.pixel_out_of_bounds(x, y):
        return
    
    if a < threshold:
        return

    try:
        pixel = self.__pixels[int(y)][int(x)]
        pixel.red = r
        pixel.green = g
        pixel.blue = b
    except Exception:
        pass


In [ ]:
Canvas.SetPixel = _NewSetPixel

In [ ]:
from RGBMatrixEmulator.emulators.canvas import Canvas

In [ ]:
pixel_index = 0
pixels = [pixel for pixel in image.getdata()]

for y in range(0, image.height):
    for x in range(0, image.width):
        try:
            self.SetPixel(x + offset_x, y + offset_y, *pixels[pixel_index])
        except Exception:
            pass

        pixel_index += 1